In [17]:
import sys
import os
import glob
import librosa
import librosa.display
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from keras.utils import to_categorical

N_MELS = 256
MEL_SPEC_FRAME_SIZE = 1024

ImportError: cannot import name 'to_categorical' from 'keras.utils' (C:\Users\natha\Anaconda3\lib\site-packages\keras\utils\__init__.py)

In [2]:
#Obtaining the data
def get_speaker_roots_in_data_path(datapath='accents'):
    speaker_list = []
    full_paths = []
    real_speakers = []
    accent_subfolders = [f.path for f in os.scandir(datapath) if f.is_dir()]
    for accent in accent_subfolders:
        for gender in ['female','male']:
            speaker_folders = os.listdir(os.path.join(accent, gender))
            for speaker in speaker_folders:
                if not speaker.startswith('.'):
                    passage_folder = os.listdir(os.path.join(accent, gender, speaker))
                    speaker_list.append(os.path.join(accent,gender, speaker))
                    for passage in passage_folder:
                        full_paths.append(os.path.join(accent, gender, speaker, passage))
                        real_speakers.append(speaker)
    return speaker_list, full_paths, real_speakers

speaker_list, all_paths, all_speakers = get_speaker_roots_in_data_path()


In [3]:
def get_wav_files_in_path(datapath):
    files = os.listdir(datapath)
    files_wav = [i for i in files if i.endswith ('.wav')]
    return files_wav


wav_files = []
for currPath in speaker_list:
    curr_wavFiles = get_wav_files_in_path(currPath)

    for i in curr_wavFiles:
        wav_files.append(i)

In [4]:
#Obtaining MFCCS
def mel_spectrogram(wavfile_path, chunk_seconds=None, plot=False):
    sig, fs = librosa.load(wavfile_path, sr=16000)
    # mel_spec_frame_size = int(fs * 0.03)

    # normalize between [-1,1]
    sig /= np.max(np.abs(sig), axis=0)

    if not chunk_seconds:
        chunk_samples = len(sig)
    else:
        # get first few seconds
        chunk_samples = fs * chunk_seconds

    melspec = librosa.feature.melspectrogram(y=sig[0:0 + chunk_samples],
                                             sr=fs,
                                             center=True,
                                             n_fft=MEL_SPEC_FRAME_SIZE,
                                             hop_length=int(MEL_SPEC_FRAME_SIZE / 2),
                                             n_mels=N_MELS)
    melspec = librosa.power_to_db(melspec, ref=1.0)
    
    return melspec, fs

def mfcc_coefficients(wavfile_path, filters=20):
    melspec, fs = mel_spectrogram(wavfile_path=wavfile_path, chunk_seconds=3, plot=False)
    mfccs = librosa.feature.mfcc(S=melspec,n_mfcc=filters)

    return mfccs

# mfccs = mfcc_coefficients(wavfile_path=all_paths[0], filters=40)
# print()
mfccs = []
speaker_full = []
cnt = 1
for currPath in all_paths:
    print(cnt)
    cnt+=1
    mfcc = mfcc_coefficients(wavfile_path=currPath, filters=40)
    mfccs.append(mfcc)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [5]:
#Splitting the data
le = LabelEncoder()
training_x, validation_x, testing_x = np.split(mfccs, [int(.6*len(mfccs)), int(.8*len(mfccs))])
print(training_x.shape)

train_y = all_speakers[:int(len(all_speakers) * 0.6)]
train_y_labels = le.fit_transform(train_y)

print(train_y_labels.shape)
val_y = all_speakers[int(len(mfccs) * 0.6):int(len(all_speakers) * 0.8)]
val_y_labels = le.fit_transform(val_y)

test_y = all_speakers[int(len(all_speakers) * 0.8):]
test_y_labels = le.fit_transform(test_y)


(513, 40, 94)
(513,)


In [16]:
#The training model
from keras.datasets import cifar10
from keras.utils import to_categorical

num_classes = 285
y_train_1h = to_categorical(train_y_labels, num_classes=num_classes)
y_val_1h = to_categorical(val_y_labels, num_classes=num_classes)
y_test_1h = to_categorical(test_y_labels, num_classes=num_classes)

print("x")
print(validation_x.shape)
print("y")
print(y_val_1h.shape)

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.regularizers import l1

model = Sequential()
model.add(LSTM(128, input_shape=(training_x.shape[1], training_x.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.add(Dense(64, activation='relu', input_shape=(input_shape,), kernel_regularizer=l1(0.01)))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(training_x, y_train_1h, batch_size=32, epochs=100, validation_data = (validation_x, y_val_1h))

ImportError: cannot import name 'to_categorical' from 'keras.utils' (C:\Users\natha\Anaconda3\lib\site-packages\keras\utils\__init__.py)